# Custom Components

This is a guide on how to build a simple app and custom component spec
and launch it via two different schedulers.

See the [Quickstart Guide](quickstart.md) for installation and basic usage.

## Hello World

Lets start off with writing a simple "Hello World" python app. This is just a
normal python program and can contain anything you'd like.

<div class="admonition note">
<div class="admonition-title">Note</div>
This example uses Jupyter Notebook `%%writefile` to create local files for
example purposes. Under normal usage you would have these as standalone files.
</div>

In [1]:
%%writefile my_app.py

import sys
import argparse

def main(user: str) -> None:
    print(f"Hello, {user}!")

if __name__ == "__main__":
    parser = argparse.ArgumentParser(
        description="Hello world app"
    )
    parser.add_argument(
        "--user",
        type=str,
        help="the person to greet",
        required=True,
    )
    args = parser.parse_args(sys.argv[1:])

    main(args.user)

Writing my_app.py


Now that we have an app we can write the component file for it. This
function allows us to reuse and share our app in a user friendly way.

We can use this component from the `torchx` cli or programmatically as part of a
pipeline.

In [2]:
%%writefile my_component.py

import torchx.specs as specs

def greet(user: str, image: str = "my_app:latest") -> specs.AppDef:
    return specs.AppDef(
        name="hello_world",
        roles=[
            specs.Role(
                name="greeter",
                image=image,
                entrypoint="python",
                args=[
                    "-m", "my_app",
                    "--user", user,
                ],
            )
        ],
    )

Writing my_component.py


We can execute our component via `torchx run`. The
`local_cwd` scheduler executes the component relative to the current directory.

In [3]:
%%sh
torchx run --scheduler local_cwd my_component.py:greet --user "your name"

torchx 2023-08-14 19:14:05 INFO     Tracker configurations: {}


torchx 2023-08-14 19:14:05 INFO     Log directory not set in scheduler cfg. Creating a temporary log dir that will be deleted on exit. To preserve log directory set the `log_dir` cfg option


torchx 2023-08-14 19:14:05 INFO     Log directory is: /tmp/torchx_neqbwku5


torchx 2023-08-14 19:14:05 INFO     Waiting for the app to finish...


greeter/0 Hello, your name!


torchx 2023-08-14 19:14:06 INFO     Job finished: SUCCEEDED


local_cwd://torchx/hello_world-pdcb9nkfgx9g4c


If we want to run in other environments, we can build a Docker container so we
can run our component in Docker enabled environments such as Kubernetes or via
the local Docker scheduler.

<div class="admonition note">
<div class="admonition-title">Note</div>
This requires Docker installed and won't work in environments such as Google
Colab. If you have not done so already follow the install instructions on:
[https://docs.docker.com/get-docker/](https://docs.docker.com/get-docker/)</a>
</div>

In [4]:
%%writefile Dockerfile.custom

FROM ghcr.io/pytorch/torchx:0.1.0rc1

ADD my_app.py .

Writing Dockerfile.custom


Once we have the Dockerfile created we can create our docker image.

In [5]:
%%sh
docker build -t my_app:latest -f Dockerfile.custom .

#0 building with "default" instance using docker driver


#1 [internal] load build definition from Dockerfile.custom


#1 transferring dockerfile: 99B done


#1 DONE 0.0s


#2 [internal] load .dockerignore


...

#7 exporting layers


#7 exporting layers 2.4s done


#7 writing image sha256:159877e7d0af4374af6e91c1b4da5150c1095974633e2df95e599141af13b398 done


#7 naming to docker.io/library/my_app:latest done


#7 DONE 2.4s


We can then launch it on the local scheduler.

In [6]:
%%sh
torchx run --scheduler local_docker my_component.py:greet --image "my_app:latest" --user "your name"

torchx 2023-08-14 19:15:30 INFO     Tracker configurations: {}


torchx 2023-08-14 19:15:30 INFO     Checking for changes in workspace `file:///home/runner/work/torchx/torchx/docs/source`...


torchx 2023-08-14 19:15:30 INFO     To disable workspaces pass: --workspace="" from CLI or workspace=None programmatically.


torchx 2023-08-14 19:15:30 INFO     Workspace `file:///home/runner/work/torchx/torchx/docs/source` resolved to filesystem path `/home/runner/work/torchx/torchx/docs/source`


torchx 2023-08-14 19:15:31 WARNING  failed to pull image my_app:latest, falling back to local: 404 Client Error for http+docker://localhost/v1.42/images/create?tag=latest&fromImage=my_app: Not Found ("pull access denied for my_app, repository does not exist or may require 'docker login': denied: requested access to the resource is denied")


...

torchx 2023-08-14 19:15:36 INFO     Built new image `sha256:8b39e1e8da89c1a5c2069f3f67adec5388aafdcafba3c5406383e96fdd0d2625` based on original image `my_app:latest` and changes in workspace `file:///home/runner/work/torchx/torchx/docs/source` for role[0]=greeter.


torchx 2023-08-14 19:15:38 INFO     Waiting for the app to finish...


greeter/0 Hello, your name!


torchx 2023-08-14 19:15:39 INFO     Job finished: SUCCEEDED


local_docker://torchx/hello_world-mzdqdhk72f7mpc


If you have a Kubernetes cluster you can use the [Kubernetes scheduler](schedulers/kubernetes.rst) to launch
this on the cluster instead.


```sh
$ docker push my_app:latest
$ torchx run --scheduler kubernetes my_component.py:greet --image "my_app:latest" --user "your name"
```

## Builtins

TorchX also provides a number of builtin components with premade images. You can discover
them via:

In [7]:
%%sh
torchx builtins

Found 11 builtin components:


  1. dist.ddp


  2. dist.spmd


  3. metrics.tensorboard


  4. serve.torchserve


...

  7. utils.copy


  8. utils.echo


  9. utils.python


 10. utils.sh


 11. utils.touch


You can use these either from the CLI, from a pipeline or programmatically like
you would any other component.

In [8]:
%%sh
torchx run utils.echo --msg "Hello :)"

torchx 2023-08-14 19:15:40 INFO     Tracker configurations: {}


torchx 2023-08-14 19:15:40 INFO     Checking for changes in workspace `file:///home/runner/work/torchx/torchx/docs/source`...


torchx 2023-08-14 19:15:40 INFO     To disable workspaces pass: --workspace="" from CLI or workspace=None programmatically.


torchx 2023-08-14 19:15:40 INFO     Workspace `file:///home/runner/work/torchx/torchx/docs/source` resolved to filesystem path `/home/runner/work/torchx/torchx/docs/source`


torchx 2023-08-14 19:17:00 INFO     Building workspace docker image (this may take a while)...


torchx 2023-08-14 19:17:08 INFO     Built new image `sha256:b1d6f55912cc8b15e0f1b666eac7f9678e91de886113c5da5f687d7889a64ea8` based on original image `ghcr.io/pytorch/torchx:0.6.0dev0` and changes in workspace `file:///home/runner/work/torchx/torchx/docs/source` for role[0]=echo.


torchx 2023-08-14 19:17:08 INFO     Waiting for the app to finish...


torchx 2023-08-14 19:17:08 INFO     Job finished: SUCCEEDED


echo/0 Hello :)


local_docker://torchx/echo-tdqlcmclnfxbzc
